In [1]:
import pandas as pd
import numpy as np
import zipfile
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import datetime
import urllib.request, urllib.error
import os.path
import datetime

In [2]:
all_deaths_path = "data/sco_covid_data/custom_data/all_deaths_lad_18.csv"
covid_deaths_path = "data/sco_covid_data/custom_data/covid_deaths_lad_18.csv"
loc_deaths_all_path = "data/sco_covid_data/custom_data/deaths_by_location_all_lad_18.csv"
loc_deaths_corona_path = "data/sco_covid_data/custom_data/deaths_by_location_corona_lad_18.csv"
path = "https://www.nrscotland.gov.uk/files//statistics/covid19/"

In [3]:
data = [['S12000033', 'Aberdeen City'],
  ['S12000034', 'Aberdeenshire'],
  ['S12000041', 'Angus council'],
  ['S12000035', 'Argyll and Bute Council'],
  ['S12000036', 'Edinburgh'],
  ['S12000005', 'Clackmannanshire'],
  ['S12000006', 'Dumfries and Galloway'],
  ['S12000042', 'Dundee City Council'],
  ['S12000008', 'East Ayrshire Council'],
  ['S12000045', 'East Dunbartonshire Council'],
  ['S12000010', 'East Lothian Council'],
  ['S12000011', 'East Renfrewshire Council'],
  ['S12000014', 'Falkirk'],
  ['S12000015', 'Fife'],
  ['S12000046', 'Glasgow City'],
  ['S12000017', 'Highland Council'],
  ['S12000018', 'Inverclyde'],
  ['S12000019', 'Midlothian'],
  ['S12000020', 'Moray'],
  ['S12000013', 'Na h-Eileanan an Iar'],
  ['S12000021', 'North Ayrshire Council'],
  ['S12000044', 'North Lanarkshire'],
  ['S12000023', 'Orkney'],
  ['S12000024', 'Perth and Kinross'],
  ['S12000038', 'Renfrewshire'],
  ['S12000026', 'Scottish Borders'],
  ['S12000027', 'Shetland Islands'],
  ['S12000028', 'South Ayrshire Council'],
  ['S12000029', 'South Lanarkshire'],
  ['S12000030', 'Stirling'],
  ['S12000039', 'West Dunbartonshire Council'],
  ['S12000040', 'West Lothian']]

In [4]:
def get_week_num():
    return datetime.date.today().isocalendar()[1]

In [5]:
get_week_num()

21

In [6]:
'''NEED TO REFACTOR THIS'''
def get_covid_deaths_df(path):
            
    # Try get get the most recent data
    try:
        url = path + "covid-deaths-data-week-%d.zip" % (get_week_num()-1)
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_covid_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-19_Table 1 - COVID deaths.csv'),header = 3, encoding='unicode-escape')
        df_all_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-19_Table 2 - All deaths.csv'),header = 3, encoding='unicode-escape')
        
        df_covid_deaths_trimmed = df_covid_deaths.iloc[48:80, 1:get_week_num()+1]
        df_all_deaths_trimmed = df_all_deaths.iloc[50:82, 1:get_week_num()+1]

        
        return df_all_deaths_trimmed, df_covid_deaths_trimmed
    
    # If error, then get last week response
    except urllib.error.HTTPError as e:
        url = path + "covid-deaths-data-week-%d.zip" % (get_week_num()-2)
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_covid_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-19_Table 1 - COVID deaths.csv'),header = 3, encoding='unicode-escape')
        
        idx_start = np.where(df_covid_deaths['Unnamed: 1'] == 'Aberdeen City')[0][0]
        idx_end = idx_start + 32
        df_covid_deaths_trimmed = df_covid_deaths.iloc[idx_start:idx_end,1:get_week_num()]
        df_covid_deaths_trimmed = df_covid_deaths_trimmed.set_index('Unnamed: 1')
        
        return df_covid_deaths_trimmed

In [7]:
def get_death_locations_df(path, week):
            
    # Try get get the most recent data
    try:
        url = path + "covid-deaths-data-week-%d.zip" % week
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_location = pd.read_csv(zipfile.open('covid-deaths-data-week-%d_Table 3 - deaths by location.csv' % week), header=3, encoding='unicode-escape')
        idx_start = np.where(df_location['Unnamed: 0'] == 'Aberdeen City')[0][0]
        idx_end = idx_start + 32
        df_location_trimmed = df_location.iloc[idx_start:idx_end, :-2]

        
        return df_location_trimmed
    
    # If error, then get last week response
    except urllib.error.HTTPError as e:
        print(week)
        url = path + "covid-deaths-data-week-%d.zip" % week-1
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        
        df_location = pd.read_csv(zipfile.open('covid-deaths-data-week-%d_Table 3 - deaths by location.csv' % week), header=3, encoding='unicode-escape')

        idx_start = np.where(df_location['Unnamed: 0'] == 'Aberdeen City')[0][0]
        print(idx_start)
        idx_end = idx_start + 32
        df_location_trimmed = df_location.iloc[idx_start:idx_end, :-2]
        return df_location_trimmed

In [8]:
df_death_locations_trimmed = get_death_locations_df(path, 19)
df_death_locations_trimmed

,Unnamed: 0,Care\nHome,Home / Non-institution,Hospital,Other\ninstitution3,All locations,Unnamed: 6,Care\nHome.1,Home / Non-institution.1,Hospital.1,Other\ninstitution3.1,All locations.1
21,Aberdeen City,40,5.0,33,0.0,78,NaN,238,237,443,0.0,918
22,Aberdeenshire,51,11.0,37,0.0,99,NaN,325,322,485,2.0,"1,134"
23,Angus,33,4.0,17,0.0,54,NaN,164,186,242,0.0,592
24,Argyll and Bute,22,8.0,28,0.0,58,NaN,106,188,201,0.0,495
25,City of Edinburgh,191,16.0,114,0.0,321,NaN,733,517,823,0.0,"2,073"
26,Clackmannanshire,26,1.0,8,0.0,35,NaN,98,74,100,2.0,274
27,Dumfries and Galloway,9,1.0,36,0.0,46,NaN,165,252,404,0.0,821
28,Dundee City,55,13.0,56,0.0,124,NaN,194,244,391,0.0,829
29,East Ayrshire,24,5.0,32,0.0,61,NaN,132,205,308,1.0,646
30,East Dunbartonshire,55,4.0,29,0.0,88,NaN,198,145,207,0.0,550


In [9]:
df_death_locations_trimmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 21 to 52
Data columns (total 12 columns):
Unnamed: 0                  32 non-null object
Care
Home                   32 non-null object
Home / Non-institution      32 non-null float64
Hospital                    32 non-null object
Other
institution3          32 non-null float64
All locations               32 non-null object
Unnamed: 6                  0 non-null float64
Care
Home.1                 32 non-null object
Home / Non-institution.1    32 non-null object
Hospital.1                  32 non-null object
Other
institution3.1        32 non-null float64
All locations.1             32 non-null object
dtypes: float64(4), object(8)
memory usage: 3.1+ KB


In [10]:
def get_data_for_location(df_locations_trimmed, area_name):
    
    df_area = df_locations_trimmed.set_index('Unnamed: 0')
    df_area_solo = df_area.loc[[area_name]]
    df_area_solo2 = df_area_solo.iloc[:,0:3]
    df_transpose = df_area_solo2.transpose()
    df_transpose = df_transpose.reset_index()
    df_transpose = df_transpose.rename(columns={'index':'Country', area_name:'Value'})
    
    return df_transpose

In [11]:
lad_dict = {"S12000033":"Aberdeen City","S12000034":"Aberdeenshire","S12000041":"Angus","S12000035":"Argyll and Bute","S12000036":"City of Edinburgh","S12000005":"Clackmannanshire","S12000006":"Dumfries and Galloway","S12000042":"Dundee City","S12000008":"East Ayrshire","S12000045":"East Dunbartonshire","S12000010":"East Lothian","S12000011":"East Renfrewshire","S12000014":"Falkirk","S12000015":"Fife","S12000046":"Glasgow City","S12000017":"Highland","S12000018":"Inverclyde","S12000019":"Midlothian","S12000020":"Moray","S12000013":"Na h-Eileanan Siar","S12000021":"North Ayrshire","S12000044":"North Lanarkshire","S12000023":"Orkney Islands","S12000024":"Perth and Kinross","S12000038":"Renfrewshire","S12000026":"Scottish Borders","S12000027":"Shetland Islands","S12000028":"South Ayrshire","S12000029":"South Lanarkshire","S12000030":"Stirling","S12000039":"West Dunbartonshire","S12000040":"West Lothian"}

In [12]:
def populate_location_deaths(df_locations_trimmed, data_dict, root_path):
    
    for x in data_dict.keys():
        df_area = get_data_for_location(df_locations_trimmed, data_dict.get(x))
        path = root_path + f"{x}.csv"
        print(path)
        current_dir = os.getcwd()
        curr = os.path.dirname(os.getcwd())
        #print(curr)
        df_area.to_csv(current_dir + path)

In [13]:
populate_location_deaths(df_death_locations_trimmed, lad_dict, "/data/lad/loc_deaths/")

/data/lad/loc_deaths/S12000033.csv
/data/lad/loc_deaths/S12000034.csv
/data/lad/loc_deaths/S12000041.csv
/data/lad/loc_deaths/S12000035.csv
/data/lad/loc_deaths/S12000036.csv
/data/lad/loc_deaths/S12000005.csv
/data/lad/loc_deaths/S12000006.csv
/data/lad/loc_deaths/S12000042.csv
/data/lad/loc_deaths/S12000008.csv
/data/lad/loc_deaths/S12000045.csv
/data/lad/loc_deaths/S12000010.csv
/data/lad/loc_deaths/S12000011.csv
/data/lad/loc_deaths/S12000014.csv
/data/lad/loc_deaths/S12000015.csv
/data/lad/loc_deaths/S12000046.csv
/data/lad/loc_deaths/S12000017.csv
/data/lad/loc_deaths/S12000018.csv
/data/lad/loc_deaths/S12000019.csv
/data/lad/loc_deaths/S12000020.csv
/data/lad/loc_deaths/S12000013.csv
/data/lad/loc_deaths/S12000021.csv
/data/lad/loc_deaths/S12000044.csv
/data/lad/loc_deaths/S12000023.csv
/data/lad/loc_deaths/S12000024.csv
/data/lad/loc_deaths/S12000038.csv
/data/lad/loc_deaths/S12000026.csv
/data/lad/loc_deaths/S12000027.csv
/data/lad/loc_deaths/S12000028.csv
/data/lad/loc_deaths

In [104]:
df_tester = get_data_for_location(df_death_locations_trimmed, 'Aberdeen City')
df_tester

Unnamed: 0,Country,Value
0,Care\nHome,22
1,Home / Non-institution,3
2,Hospital,23


In [106]:
df_tester.to_csv('tester2.csv')